# X_Scraper end-to-end walkthrough

This notebook demonstrates how to install dependencies, configure the scraper, fetch data, clean it, run sentiment analysis, and generate reports. Edit the configuration values to point at your own API endpoint or adjust thresholds.

In [ ]:
%pip install -q --disable-pip-version-check pandas requests vaderSentiment matplotlib

## Imports and helpers

Run the next cell to load the pipeline components. The package lives in this repository, so you can modify it locally and re-run these cells.

In [ ]:
from pathlib import Path

import pandas as pd

from x_scraper import ScraperConfig, XScraper

## Configuration

Adjust the values below to control the run:

- `source_url`: API endpoint or JSON feed to scrape.
- `limit`: Maximum number of records to fetch.
- `min_chars`: Minimum character length retained after cleaning.
- `output_dir`: Where CSV outputs will be written.
- `detail_filename` / `report_filename`: Override output filenames if needed.

In [ ]:
config = ScraperConfig(
    source_url="https://jsonplaceholder.typicode.com/posts",  # swap in your endpoint
    limit=25,
    min_chars=40,
    output_dir=Path("artifacts"),
)
config

## Scrape

The cell below downloads posts from the configured `source_url`. For a different dataset, update the configuration above or inject your own DataFrame if you already have raw content.

In [ ]:
pipeline = XScraper(config)
raw_posts = pipeline.scrape()
raw_posts.head()

## Clean the text

The cleaning step collapses whitespace and trims short entries. Increase `min_chars` if you see noisy, low-value rows, or set it to `0` to keep everything.

In [ ]:
clean_posts = pipeline.clean(raw_posts)
clean_posts.head()

## Sentiment analysis

VADER sentiment is applied to the cleaned text. The `sentiment_score` is the compound score in `[-1, 1]` and the `sentiment` label is derived from the thresholds used by the analyzer.

In [ ]:
sentiment_posts = pipeline.score_sentiment(clean_posts)
sentiment_posts[["id", "sentiment_score", "sentiment"]].head()

## Reporting

Both detailed and aggregated reports are saved under `output_dir`:

- `scraped_posts.csv`: Each cleaned post with sentiment columns.
- `sentiment_report.csv`: Counts of positive/neutral/negative content.

You can load the outputs into another tool or tweak the DataFrame directly below.

In [ ]:
report_paths = pipeline.generate_reports(sentiment_posts)
report_paths

In [ ]:
summary = sentiment_posts["sentiment"].value_counts().rename_axis("sentiment").reset_index(name="count")
summary

## Interpreting the outputs

- Review the `summary` table to see whether your dataset skews positive, neutral, or negative.
- Inspect the `scraped_posts.csv` file to confirm that cleaning preserved the information you care about.
- If you need stricter filtering, raise `min_chars` or add extra cleaning rules inside `x_scraper/pipeline.py`.
- To plug in a different data source, change `source_url` to point at your API, adjust field parsing in `_merge_fields`, and re-run the notebook.